In [1]:
# Earth Engine Python API
import ee
ee.Initialize()

In [2]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.10.0


In [3]:

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [95]:
# Silk board junction.
geometry = ee.Geometry.Point(77.6192442, 12.9118799).buffer(10000)

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filterDate('2015-01-01', '2015-12-01')\
               .filterBounds(geometry)\
               .sort('CLOUD_COVER').limit(5)\
               .select(['B4', 'B3', 'B2'])

median = collection.median()
roi = median.clip(geometry)

# dicc = {
#     'median' : median.getMapId()
# }

# center = [12.9118799, 77.6192442]
# Mapdisplay(center, dicc, zoom_start= 13)

In [96]:
roi.getThumbURL(params={'min': 0,'max': 0.3, 'dimensions':512, 'format':'jpg'})

'https://earthengine.googleapis.com/api/thumb?thumbid=9c58ddf4da734432b37ea22ec1fb3624&token=41bdb2438ac9977ef7fe82a677dadbcd'

In [97]:
# Silk board junction.
geometry = ee.Geometry.Point(77.6192442, 12.9118799).buffer(10000)

def maskS2clouds(image):
  qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000);

collection = ee.ImageCollection('COPERNICUS/S2')\
               .filterDate('2015-06-01', '2015-12-31')\
               .filterBounds(geometry)\
               .sort('CLOUDY_PIXEL_PERCENTAGE').limit(5)\
               .map(maskS2clouds)\
               .select(['B4', 'B3', 'B2'])

               

count = collection.size()
print(count.getInfo())

median = collection.median()
roi = median.clip(geometry)
# dicc = {
#     'median' : roi.getMapId({'min': 0, 'max': 0.3})
# }

# center = [12.9118799, 77.6192442]
# Mapdisplay(center, dicc, zoom_start= 13)

5


In [98]:
roi.getThumbURL(params={'min': 0,'max': 0.3, 'dimensions':512, 'format':'jpg'})

'https://earthengine.googleapis.com/api/thumb?thumbid=6a58fd8c1bd4e321861dd74b2ea2a23d&token=9648ed5d7a839e4eb32c92add5e479bc'

In [88]:
qa = collection.select('QA60');

# Bits 10 and 11 are clouds and cirrus, respectively.
cloudBitMask = 1 << 10;
cirrusBitMask = 1 << 11;

mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

AttributeError: 'ImageCollection' object has no attribute 'bitwiseAnd'

In [99]:
url = roi.getThumbURL(params={'dimensions':1024, 'format':'jpg'})

In [101]:
import requests

In [102]:
r = requests.get(url)

In [105]:
with open('sample.jpeg', 'wb') as f:
    f.write(r.content)

In [60]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': median,
    'description': 'test-image-100',
    'folder':'quality-of-life',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [62]:
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: 5LFHXUZG3JFJDTK32LZ3MCF3).
Polling for task (id: 5LFHXUZG3JFJDTK32LZ3MCF3).


In [35]:
# Load Landsat 5 data, filter by date and bounds.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T2').filterDate('1987-01-01', '1990-05-01').filterBounds(ee.Geometry.Point(25.8544, -18.08874))

# Also filter the collection by the IMAGE_QUALITY property.
filtered = collection.filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
badComposite = ee.Algorithms.Landsat.simpleComposite(collection, 75, 3)
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

dicc = {
    'Bad composite' : badComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5}),
    'Good composite': goodComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5})
}

# Display the results
center = [-18.08874, 25.8544]
Mapdisplay(center, dicc, zoom_start= 13)